<a href="https://colab.research.google.com/github/laralausen/laralausen.github.io/blob/master/training_and_playgame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt

from DQN_Agent import DQNSolver
from qcartpole import QuantumCartPoleEnv


env = QuantumCartPoleEnv()
observation_space = env.observation_space
action_space = env.action_space
np.random.seed(12345)
random.seed(12345)

# Create a new random network
def training():

    dqn_agent = DQNSolver(observation_space, action_space, gamma = 0.9999, learning_rate=0.001, hidden_layer_1_size=32, hidden_layer_2_size=64) 
    scores =[]
    best_score = 0
    actions = []

    
    for i in range(200):
        state = env.reset()
        done = False
        score = 0
        action = 0
        position = 0
        momentum = 0
        variance = 0 

        while not done:
            
            action = dqn_agent.get_action(state)
            new_state, reward, done, _ = env.step(action) 
            score += reward
            dqn_agent.remember(state, action, reward, new_state, done)
            state = new_state
            loss = dqn_agent.learn()
          
            actions.append(action)
        
        
        scores.append(score)
        avg_score = np.mean(scores[max(0,i-100):(i+1)])
        print('episode ', i,'score %.2f' %score,
              'average score %.2f' % avg_score)
        

        if( score == best_score ):
            best_score = score 
            dqn_agent.save("best-network")
            print("network saved")
            
                    
    return scores

all_rounds_scores = []  

for i in range(1):
    scores = training()
    all_rounds_scores.append(scores)



def play_game(networkfile, printing=True):
    env = QuantumCartPoleEnv()
    observation_space = env.observation_space
    action_space = env.action_space
    dqn_agent = DQNSolver(observation_space, action_space, gamma = 0.9999, learning_rate=0.0001, hidden_layer_1_size=32, hidden_layer_2_size=64)
    dqn_agent.load(networkfile)
    run = 0
    state = env.reset()
    done = False
    score = 0
    actions = []
    positions = []
    momenta = []
    variances = []
    
    while not done:
        # Act
        action = dqn_agent.get_action(state, use_exploration=False)   
        momentum = env.compute_exp_p()
        position = env.compute_exp_x()
        variance = env.compute_variance()

        actions.append(action)
        momenta.append(momentum)
        positions.append(position)
        variances.append(variance)

        new_state, reward, done, _ = env.step(action)
        score += reward
        state = new_state

        if done and printing == True:
            print("Finished with score: " + str(score))
            break
            
        elif done and printing == False:
            break
            



play_game("best-networkmodel1")